In [1]:
#mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os

In [3]:
#changing to directory location where we have csv file
os.chdir('/content/drive/MyDrive/Deep_Learning/yolov4(object_detection)/Dataset_and_Annotations/csv_folder')

In [7]:
class_data= pd.read_csv('class-descriptions-boxable.csv', header=None)

In [8]:
class_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [9]:
classes= ['Person','Car']

In [11]:
class_strings=[]

for i in classes:
  req_classes= class_data.loc[class_data[1]==i]
  string= req_classes.iloc[0][0]
  print(string)
  class_strings.append(string)
print (class_strings)

/m/01g317
/m/0k4j
['/m/01g317', '/m/0k4j']


In [16]:
#getting only required columns from annotation csv file

annotation_data= pd.read_csv('train-annotations-bbox.csv', usecols=['ImageID','LabelName' ,'XMin', 'YMax', 'XMax', 'YMin'])

In [17]:
annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [18]:
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_strings)].copy()

In [19]:
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [20]:
#Adding new columns that are required for YOLO format
filtered_class_data['classNumber']=''
filtered_class_data['center x']=''
filtered_class_data['center y']=''
filtered_class_data['width']=''
filtered_class_data['height']=''

In [21]:
for i in range(len(class_strings)):
  filtered_class_data.loc[filtered_class_data['LabelName']== class_strings[i], 'classNumber']=i 

  #calculating center x and center y
  filtered_class_data['center x'] = (filtered_class_data['XMax'] + filtered_class_data['XMin'])/2
  filtered_class_data['center y'] = (filtered_class_data['YMax'] + filtered_class_data['YMin'])/2

  #calculating width and height
  filtered_class_data['width'] = (filtered_class_data['XMax'] - filtered_class_data['XMin'])
  filtered_class_data['height'] = (filtered_class_data['YMax'] - filtered_class_data['YMin'])


In [22]:
yolo_values= filtered_class_data.loc[: , ['ImageID', 'classNumber' , 'center x', 'center y', 'width', 'height']].copy()

In [23]:
yolo_values.head()

,ImageID,classNumber,center x,center y,width,height
0,000002b66c9c498e,0,0.103906,0.367969,0.182812,0.439062
1,000002b66c9c498e,0,0.150782,0.831251,0.251563,0.234375
2,000002b66c9c498e,0,0.231249,0.394531,0.159375,0.392188
3,000002b66c9c498e,0,0.342969,0.788282,0.173438,0.273437
4,000002b66c9c498e,0,0.302343,0.310938,0.089063,0.150000


In [24]:
image_path= '/content/drive/MyDrive/Deep_Learning/yolov4(object_detection)/Dataset_and_Annotations/Dataset/train/Person_Car'

In [25]:
os.chdir(image_path)

In [28]:
for current_dir , dirs , files in os.walk('.'):
  for f in files:
    #iterating through all the files
    if f.endswith('.jpg'):
      #extract only title of image file so that corresponding txt file can be generated with same name as per yolo
      image_title= f[:-4]
      #get matched data
      yolo_file= yolo_values.loc[yolo_values['ImageID']== image_title]

      df= yolo_file.loc[: , ['classNumber' , 'center x' , 'center y', 'width', 'height']].copy()

      save_path = image_path + '/' + image_title + '.txt'

      df.to_csv(save_path, header=False, index= False, sep=' ')